In [3]:
import boto3
import subprocess
import json
import os
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed

/var/folders/sb/ph4rxk6j27v6qfqt_n53_bzm0000gq/T/ipykernel_49829/723324291.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def set_aws_credentials(profile, region_name='us-east-1'):
    result = subprocess.run(f"aws-vault exec {profile} --json", shell=True, capture_output=True)
    credentials = json.loads(result.stdout)

    # Create a session with the retrieved credentials
    session = boto3.session.Session(
        aws_access_key_id=credentials['AccessKeyId'],
        aws_secret_access_key=credentials['SecretAccessKey'],
        aws_session_token=credentials['SessionToken'],
        region_name=region_name        
    )

    return session

# Use the function with your profile to get a session
aws_session = set_aws_credentials('acl-production', 'eu-central-1')

# Create clients using the session
sts_client = aws_session.client('sts')
logs_client = aws_session.client('logs')

# Example usage of the clients
account_id = sts_client.get_caller_identity()["Account"]
print("Current AWS Account ID:", account_id)

Current AWS Account ID: 707785685172


# List LogGroups

In [5]:
# list log groups

def list_log_groups(logs_client):
    paginator = logs_client.get_paginator('describe_log_groups')
    response_iterator = paginator.paginate()
    log_groups = []
    for response in response_iterator:
        log_groups += response['logGroups']
    return log_groups

log_groups = list_log_groups(logs_client)
print("Number of log groups:", len(log_groups))

print (json.dumps(log_groups, indent=4))

Number of log groups: 455
[
    {
        "logGroupName": "/aws/api-gateway-v2/impact-reports-report-dispatcher",
        "creationTime": 1662603567617,
        "retentionInDays": 365,
        "metricFilterCount": 1,
        "arn": "arn:aws:logs:eu-central-1:707785685172:log-group:/aws/api-gateway-v2/impact-reports-report-dispatcher:*",
        "storedBytes": 30199,
        "kmsKeyId": "arn:aws:kms:eu-central-1:707785685172:key/ccf64012-cc83-4049-974e-38f30be2b2ad",
        "logGroupClass": "STANDARD"
    },
    {
        "logGroupName": "/aws/api-gateway-v2/importer-session-notifier-gateway",
        "creationTime": 1663771070652,
        "retentionInDays": 365,
        "metricFilterCount": 1,
        "arn": "arn:aws:logs:eu-central-1:707785685172:log-group:/aws/api-gateway-v2/importer-session-notifier-gateway:*",
        "storedBytes": 8600084,
        "kmsKeyId": "arn:aws:kms:eu-central-1:707785685172:key/ccf64012-cc83-4049-974e-38f30be2b2ad",
        "logGroupClass": "STANDARD"
   

## List log streams for a given log group



In [15]:
# list log streams
def list_log_streams(logs_client, log_group_name):
    paginator = logs_client.get_paginator('describe_log_streams')
    
    
    response_iterator = paginator.paginate(logGroupName=log_group_name)
    log_streams = []
    for response in response_iterator:
        log_streams += response['logStreams']
        print('number of log streams:', len(log_streams))
        if len(log_streams) > 1000:
            break
    return log_streams

log_streams = list_log_streams(logs_client, 'projects-main')
print("Number of log streams:", len(log_streams))

# list logstreams
print (json.dumps(log_streams, indent=4))


# list distinct logstream prefixes which are in the following format 'service/container_name'
# e.g. 'service/ingress-nginx-controller'
# the prefix should contain both the service and the container name
def list_logstream_prefixes(log_streams):
    logstream_prefixes = []
    for log_stream in log_streams:
        logstream_prefix = log_stream['logStreamName'].split('/')[0:2]
        logstream_prefix = '/'.join(logstream_prefix)
        if logstream_prefix not in logstream_prefixes:
            logstream_prefixes.append(logstream_prefix)
    return logstream_prefixes


logstream_prefixes = list_logstream_prefixes(log_streams)
print("Number of logstream prefixes:", len(logstream_prefixes))
print(logstream_prefixes)


number of log streams: 50
number of log streams: 100
number of log streams: 150
number of log streams: 200
number of log streams: 250
number of log streams: 300
number of log streams: 350
number of log streams: 400
number of log streams: 450
number of log streams: 500
number of log streams: 550
number of log streams: 600
number of log streams: 650
number of log streams: 700
number of log streams: 750
number of log streams: 800
number of log streams: 850
number of log streams: 900
number of log streams: 950
number of log streams: 1000
number of log streams: 1050
Number of log streams: 1050
[
    {
        "logStreamName": "application/TwistlockDefender/0006f3e0bfad4ff2af3162a1d1720b0e",
        "creationTime": 1700270177407,
        "arn": "arn:aws:logs:eu-central-1:707785685172:log-group:projects-main:log-stream:application/TwistlockDefender/0006f3e0bfad4ff2af3162a1d1720b0e",
        "storedBytes": 0
    },
    {
        "logStreamName": "application/TwistlockDefender/0008101de4044d059